# Wildfire Analysis Live Example

In [ ]:
import geopandas as gpd
import folium
import branca

# Get file name
from pathlib import Path

file_name = "WFIGS_Interagency_Perimeters_YTD.geojson"
wf_perim_path = Path.cwd() / file_name
wf_perims = gpd.read_file(wf_perim_path)

In [ ]:
# Load map
map = folium.Map(location=[34, -110], tiles=None, zoom_start=4)
# Add OpenStreetMap tile layer
folium.TileLayer("OpenStreetMap", name="OpenStreetMap").add_to(map)
# Add CartoDB tile layer
folium.TileLayer("CartoDB Positron", name="CartoDB Positron").add_to(map)

In [ ]:
# Load state data
state_data = gpd.read_file(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
)

cols = [
    "attr_IncidentName",
    "poly_GISAcres",
    "attr_EstimatedCostToDate",
    "geometry",
]

wf_perims["attr_EstimatedCostToDate"] = wf_perims.attr_EstimatedCostToDate.fillna(
    value=0
)
wf_perims["poly_GISAcres"] = wf_perims["poly_GISAcres"].fillna(value=0)

wf_perims = wf_perims[wf_perims["poly_GISAcres"] >= 0]
wf_perims = wf_perims[wf_perims["attr_EstimatedCostToDate"] >= 0]

# Count of fires in each state
state_data["fire_count"] = state_data.geometry.apply(
    lambda state_geom: wf_perims.geometry.intersects(state_geom).sum()
    if not wf_perims.empty
    else 0
)

# Create state-level colormap for fire count
colormap = branca.colormap.LinearColormap(
    colors=["green", "yellow", "orange", "red"],
    vmin=0,
    vmax=state_data["fire_count"].max(),
)


In [ ]:
# Add state boundaries layer
folium.GeoJson(
    state_data,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["fire_count"]),
        "color": "black",
        "weight": 1,
        "dashArray": "2,5 ",
    },
    name="States (Fire Count)",  # Layer name for the state boundaries
    popup=folium.GeoJsonPopup(fields=["name","fire_count"]),
).add_to(map)


In [ ]:
# Add wildfire perimeters layer with popups
popup = folium.GeoJsonPopup(
    fields=["attr_IncidentName", "poly_GISAcres", "attr_EstimatedCostToDate"],
    aliases=["Incident Name", "Acres", "Cost"],
    localize=True,
    labels=True,
    style="background-color: blue;",
)

folium.GeoJson(
    wf_perims,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["poly_GISAcres"]),
        "color": colormap(x["properties"]["poly_GISAcres"]),
    },
    name="Wildfire Perimeters",  # Layer name for wildfire perimeters
    popup=popup,
).add_to(map)

# Add colormap legend
colormap.caption = "Fire Count"
colormap.add_to(map)


In [ ]:
# Add layer control to toggle layers
folium.LayerControl().add_to(map)

# Display the map
map